# Hierarchical Temporal Memory

### Cortical Column <br/>
 A single layer in an HTM network is structured
as a set of mini-columns, each with a set of cells (Figure 1B).
The HTM neuron model incorporates dendritic properties of
pyramidal cells in neocortex (Spruston, 2008), where proximal
and distal dendritic segments on HTM neurons have different
functions (Figure 1C) (Hawkins and Ahmad, 2016). Patterns
detected on proximal dendrites lead to action potentials and
define the classic receptive field of the neuron. Patterns
recognized by a neuron’s distal synapses act as predictions by
depolarizing the cell without directly causing an action potential.
![avatar](img/column.png)

### HTM Neuron

HTM Neuron的distal dendrite部分形似树干<br/>
每个Segment就是树干上的一段树枝<br/>
每个Synapse就是树枝上的一个生长点<br/>
![avatar](img/neuron.png)

### Spatial Pooling
![avatar](img/pooling.png)

### Hebbian Learning

<b>Permanence value</b>: A scalar value (0.0 to 1.0) that is assigned to each synapse to indicate how permanent the
connection is. When a connection is reinforced, its permanence value is increased. Under other conditions, the
permanence value is decreased.</br>

<b>Permanence threshold</b>: If a synapse’s permanence value is above this threshold, it is considered fully connected.
Acceptable values are [0,1].</br>

<b>Synapse</b>: A junction between cells. A synapse can be in the following states:</br>
 - <b>Connected</b>   permanence is above the threshold.</br>
 - <b>Potential</b>   permanence is below the threshold.</br>
 - <b>Unconnected</b> does not have the ability to connect.

### Temporal Memory

A column is predicted if any of its cells have an active distal dendrite segment<br/>
<b>预测为正，真实为正</b><br/>
For each active column, if any cell was predicted, those predicted cells become active (lines 11-12). Each of these cells is marked
as a “winner” (line 13), making them presynaptic candidates for synapse growth in the next time step.<br/>
For each of these correctly active segments, reinforce the synapses that activated the segment, and punish the synapses that didn’t
contribute (lines 16-20). If the segment has fewer than SYNAPSE_SAMPLE_SIZE active synapses, grow new synapses to a subset
of the winner cells from the previous time step to make up the difference (lines 22 – 24).<br/>
在这种情况下，active cell全部成为winner cell<br/>
<b>预测为负，真实为正</b><br/>
If the column activation was unexpected, then each cell in the column becomes active (lines 26-27).<br/>
Select a winner cell and a learning segment for the column (lines 29-35).<br/>
If any cells have a matching segment, select the best
matching segment and its cell (lines 30-31). <br/>
Otherwise select the least used cell and grow a new segment on it (line 33-35).<br/>
On the learning segment, reinforce the synapses that partially activated the segment, and punish the synapses that didn’t
contribute (lines 40-44). Then grow new synapses to a subset of the previous time step’s winner cells (lines 46-48).<br/>
<b>预测为正，真实为负</b><br/>
When a column with matching segments doesn’t become active, punish the synapses that caused these segments to be
“matching”.<br/>
<b>预测为负，真实为负</b><br/>
Do nothing.

# Code Repo

In [38]:
import numpy as np
import math
import random

In [33]:
# * 探索代码
'''
[1 if x > 7 else 0 for x in range(0, 10)]

encoder = Encoder()
sdr = encoder.encode(0.7)
np.array(sdr).sum()
'''
print(columns[4].cells[7].column)

### SDR SCalar Encoder

In [7]:
# Encoder.
class Encoder:
    def __init__(self):
        super(Encoder, self).__init__()
    '''
    input - output from a MinMaxScalar, ranging from 0 to 1
    output- sparse distributed representation
    '''
    def encode(self, f):
        if (f <= 0 or f >= 1):
            raise Exception('range of (0, 1) exceeded')
        lower = math.floor(2000 * f)
        upper = lower + 49
        return [1 if x >= lower and x < upper else 0 for x in range(0, 2048)]

### Temoral Memory

In [36]:
# 定义参数
LEARNING_ENABLED=True
## 网络规模
NUMBER_OF_COLUMNS = 2048
NUMBER_OF_CELLS   = 32
MAX_SEGMENTS      = 256
MAX_SYNAPSES      = 256
## Synapse -> Segment
INITIAL_PERMANENCE= 0.21
CONNECTED_PERMANENCE=0
ACTIVATION_PERMANENCE=0.5
PERMANENCE_INCREMENT=0.1
PERMANENCE_DECREMENT=0.1
PREDICTED_DECREMENT=0
SYNAPSE_SAMPLE_SIZE=20
## Segment -> Column/Cell
ACTIVATION_THRESHOLD=13
LEARNING_THRESHOLD  =7

In [15]:
# 定义全局计数器
columns_ctr = 0
cell_ctr = 0
segment_ctr = 0
synapse_ctr = 0

In [39]:
# 定义对象
class Column:
    def __init__(self):
        super(Column, self).__init__()
        global columns_ctr
        self.id = columns_ctr
        columns_ctr = columns_ctr + 1
        self.cells = []
    def __str__(self):
        return ("Column%d" % (self.id))

class Cell:
    def __init__(self, column):
        super(Cell, self).__init__()
        global cell_ctr
        self.id = cell_ctr
        cell_ctr = cell_ctr + 1
        self.column = column
        self.segments = []
    def __str__(self):
        return ("Cell%d" % (self.id))

class Segment:
    def __init__(self, cell):
        super(Segment, self).__init__()
        global segment_ctr
        self.id = segment_ctr
        segment_ctr = segment_ctr + 1
        self.cell = cell
        self.synapses=[]
    def __str__(self):
        return ("Segment%d" % (self.id))

class Synapse:
    def __init__(self, presynapticCell, postSynapticSegment, permanence):
        super(Synapse, self).__init__()
        global synapse_ctr
        self.id = synapse_ctr
        synapse_ctr = synapse_ctr + 1
        self.presynapticCell = presynapticCell
        self.postSynapticSegment= postSynapticSegment
        self.permanence  = permanence
    def __str__(self):
        return ("Synapse%d" % (self.id))

In [13]:
# 定义数据结构
columns = []
active_columns = []
cells   = []
active_cells = []
winner_cells = []
segments= []
active_segments = []
matching_segments = []
synapses= []
num_active_potential_senapse = []

In [31]:
# 初始化Column/Cell
for i in range(0, NUMBER_OF_COLUMNS):
    column = Column()
    columns.append(column)
    for j in range(0, NUMBER_OF_CELLS):
        cell = Cell(column)
        column.cells.append(cell)

Data Manipulation Functions

In [40]:
def chooseRandom(seq):
    if len(seq < 1):
        raise Exception('empty list')
    return random.sample(seq, len(seq))[0]

In [41]:
def segmentsForColumn(column, segments):
    filtered = []
    for segment in segments:
        if segment.cell.column.id == column.id:
            filtered.append(segment)
    return filtered

In [42]:
def growNewSegment(cell):
    segment = Segment(cell)
    cell.segments.append(segment)
    segments.append(segment)
    return segment

In [43]:
def createNewSynapse(segment, presynapticCell, permanence):
    synapse = Synapse(presynapticCell, segment, permanence)
    segment.synapses.append(synapse)
    synapses.append(synapse)
    return synapse

Utility

In [44]:
def leastUsedCell(column):
    fewestSegments = MAX_SEGMENTS
    for cell in column.cells:
        fewestSegments = min(fewestSegments, cell.segments.length)

    leastUsedCells = []
    for cell in column.cells:
        if cell.segments.length == fewestSegments:
            leastUsedCells.append(cell)

    return chooseRandom(leastUsedCells)

In [45]:
def bestMatchingSegment(column):
    bestMatchingSegment = None
    bestScore = -1
    for segment in segmentsForColumn(column, matching_segments[t-1]):
        if num_active_potential_senapse[t-1][segment] > bestScore:
            bestMatchingSegment = segment
            bestScore = num_active_potential_senapse[t-1][segment]

    return bestMatchingSegment

In [46]:
def growSynapses(segment, newSynapseCount):
    candidates = winner_cells[t-1].clone()
    while len(candidates) > 0 and newSynapseCount > 0:
        presynapticCell = chooseRandom(candidates)
        candidates.remove(presynapticCell)

        alreadyConnected = False
        for synapse in segment.synapses:
            if synapse.pre_segment.cell == presynapticCell:
                alreadyConnected = True

        if alreadyConnected == False:
            newSynapse = createNewSynapse(segment, presynapticCell, INITIAL_PERMANENCE)
            newSynapseCount = newSynapseCount - 1

Sequential Memory

In [47]:
def activatePredictedColumn(column):
    for segment in segmentsForColumn(column, active_segments[t-1]):
        active_cells[t].append(segment.cell)
        winner_cells[t].append(segment.cell)
    if LEARNING_ENABLED:
        for synapse in segment.synapses:
            if synapse.pre_segment.cell in active_cells[t-1]:
                synapse.permanence = synapse.permanence + PERMANENCE_INCREMENT
            else:
                synapse.permanence = synapse.permanence - PERMANENCE_DECREMENT
        newSynapseCount = (SYNAPSE_SAMPLE_SIZE - num_active_potential_senapse[t-1][segment])
        growSynapses(segment, newSynapseCount)

In [48]:
def burstColumn(column):
    for cell in column.cells:
        active_cells[t].append(cell)
    
    if segmentsForColumn(column, matching_segments[t-1]).length > 0:
        learningSegment = bestMatchingSegment(column)
        winnerCell = learningSegment.cell
    else:
        winnerCell = leastUsedCell(column)
        if LEARNING_ENABLED:
            learningSegment = growNewSegment(winnerCell)

    winner_cells[t].add(winnerCell)

    if LEARNING_ENABLED:
        for synapse in learningSegment.synapses:
            if synapse.pre_segment.cell in active_cells[t-1]:
                synapse.permanence += PERMANENCE_INCREMENT
            else:
                synapse.permanence -= PERMANENCE_DECREMENT

        newSynapseCount = (SYNAPSE_SAMPLE_SIZE - num_active_potential_senapse[t-1][learningSegment])
        growSynapses(learningSegment, newSynapseCount)

In [49]:
def punishPredictedColumn(column):
    if LEARNING_ENABLED:
        for segment in segmentsForColumn(column, matching_segments[t-1]):
            for synapse in segment.synapses:
                if synapse.pre_segment.cell in active_cells[t-1]:
                    synapse.permanence -= PREDICTED_DECREMENT

HTM

In [50]:
series = [0.3, 0.5, 0.1, 0.7]
encoder = Encoder()
t = 0
for kpi in series:
    if t > 0:
        t = t + 1
    active_columns[t] = encoder.encode(kpi)
    # Evaluate the active columns against predictions. Choose a set of active cells
    for column in columns:
        if column in active_columns[t]:
            if segmentsForColumn(column, active_segments[t-1]).length > 0:
                activatePredictedColumn(column)
            else:
                burstColumn(column)
        elif segmentsForColumn(column, matching_segments[t-1]).length > 0:
            punishPredictedColumn(column)
    # Activate a set of dendrite segments
    for segment in segments:
        numActiveConnected = 0
        numActivePotential = 0
        for synapse in segment.synapses:
            if synapse.pre_segment.cell in active_cells[t]:
                if synapse.permanence >= CONNECTED_PERMANENCE:
                    numActiveConnected += 1

                if synapse.permanence >= 0:
                    numActivePotential += 1

        if numActiveConnected >= ACTIVATION_THRESHOLD:
            active_segments[t].append(segment)

        if numActivePotential >= LEARNING_THRESHOLD:
            matching_segments[t].append(segment)

        num_active_potential_senapse[t][segment] = numActivePotential

IndexError: list assignment index out of range